In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import CoolProp.CoolProp as Cool
from scipy import integrate
from scipy import interpolate
from scipy.optimize import curve_fit
from scipy.optimize import fsolve

from pymoo.util.misc import stack
from pymoo.model.problem import Problem
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.factory import get_termination
from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.operators.mixed_variable_operator import MixedVariableSampling, MixedVariableMutation, MixedVariableCrossover
from pymoo.optimize import minimize

plt.rc('font',family = 'serif',size = 12)
plt.rc('text', usetex = True)

ModuleNotFoundError: No module named 'pymoo'

In [ ]:
def fan(V,a,b,c):
    P = a*V**2+b*V + c
    return P

In [ ]:
#Properties of the water

rho_w = 997        #[kg/m3] Density (25ºC)
cp_w  = 4181       #[kJ/kg.K] Specific heat

mu_wc = Cool.PropsSI('V','T',273.15+10,'P',101325,'Water')
mu_wh = Cool.PropsSI('V','T',273.15+40,'P',101325,'Water')


Pr_wc = Cool.PropsSI('PRANDTL','T',273.15+10,'P',101325,'Water')
Pr_wh = Cool.PropsSI('PRANDTL','T',273.15+40,'P',101325,'Water')

k_wc=Cool.PropsSI('L','T',273.15+10,'P',101325,'Water')
k_wh=Cool.PropsSI('L','T',273.15+40,'P',101325,'Water')

#Properties of air at 25ºC

rho_a = 1.168      #[kg/m3] Density
mu_a  = 0.00001849 #[kg/(m.s)] Viscosity
cp_a  = 1004      #[kJ/kg.K] Specific heat
Pr_a  = 0.7281     #[-] Prandtl number

#Properties of the metals

k_w = 273 
rho_al = 2700
rho_cu = 9000
rho_wa = 1000

In [ ]:
#Fan Calculations

## 50 Hz
dP_fan = np.array([0,30,60,80])
V_fan = np.array([3200,2775,2270,1760])/3600
eta_fan = (dP_fan*V_fan)/np.array([134,142,151,158])

## 60 Hz
dP_fan = np.array([0,20,40,60])
V_fan = np.array([3540,3245,2895,2480])/3600
eta_fan = (dP_fan*V_fan)/np.array([183,189,194,200])



Coef_fan = curve_fit(fan,V_fan,dP_fan,maxfev=10000)
Coef_eta = curve_fit(fan,V_fan,eta_fan,maxfev=10000)

In [ ]:
def HEx(Data_Master):
    
    Xl,Xt,N_r,N_l,L1,Di,pf,xf,sigmaf,sigmah,pd,m_f1,Nucleos = Data_Master
    
    
    xf = 7*10**-3
    De = Di + 0.0008
    m_f = m_f1/Nucleos
    Beta = 1/pf
    Nfin = Beta*L1   
    
    dc = De + 2*sigmaf
    theta = np.arctan(pd/xf)

    L3 = (N_l*Xt)/2*Nucleos
    L2 = N_r*Xl
    N_t = N_r*N_l/2*Nucleos 

    a = ((Xt - De) - (Xt - De)*sigmaf/np.cos(theta)*Beta)/2
    b = ((Xt/2)**2+Xl**2)**0.5 - De - (Xt-De)*sigmaf/np.cos(theta)*Beta
    if a<b:
        c = 2*a
    else:
        c = 2*b
    A_min_o = ((L3/Xt-1)*c + (Xt-De)-(Xt-De)*sigmaf/np.cos(theta)*Beta)*L1 #Minimum free flow area
    
    
    A_fr = L1*L3 #Frontal area
    
    sigma = A_min_o/A_fr
    
    
    A_p = np.pi*De*(L1-sigmaf*Beta*L1/np.cos(theta))*N_t+2*(L2*L3 - np.pi*De**2*N_t/4) #Primary surface area  - exposed tubes and header plates
    A_f = 2*(L2*L3/np.cos(theta)-(np.pi*De**2/4)*N_t)*Beta*L1 + 2*L3*sigmaf*Beta*L1/np.cos(theta)
    A_s = A_p + A_f       #Total heat transfer area
    A_t = np.pi*De*L1*N_t #External tube area
    
    sigma_o = (L3*L1 - L3*sigmaf*Beta*L1)/(L3*L1) #area concentration/expansion ratio
    D_h = 4*A_min_o*L2/A_s #Hydraulic diameter
    
    #Correlation parameters
    
    c1 = -0.1707 - 1.374*((Xl/sigmaf)**(-0.493))*((pf/dc)**(-0.886))*((N_r)**(-0.143))*((pd/xf)**(-0.0296))
    c2 = 0.1714 - 0.07372*((pf/Xl)**0.25)*(np.log(A_s/A_t))*((pd/xf)**(-0.2))
    c3 = 0.426*((pf/Xt)**0.3)*(np.log(A_s/A_t))    

        
    #Pressure drop correlation
    def PressureDrop(vazao):
        u_air = vazao/(L1*L3)
        u_max = u_air/sigma
        Re = u_max*rho_a*dc/(mu_a)
        c4 = -10.2192/np.log(Re)
        f_o  = 0.05273*(Re**c2)*((pd/xf)**c3)*((pf/Xt)**c4)*((np.log(A_s/A_t))**(-2.726))*(D_h/dc)**0.1325*((N_r)**0.02305)   
        G_o = rho_a*u_max

        dP_o = G_o**2/(2*rho_a)*((f_o)*(A_s/A_min_o))
        dP_fan = fan(vazao,Coef_fan[0][0],Coef_fan[0][1],Coef_fan[0][2])
        return(dP_o-dP_fan)
    
    vazao = fsolve(PressureDrop,3000)
    u_air = vazao/(L1*L3)
    u_max = u_air/sigma
    Re = u_max*rho_a*dc/(mu_a)
    Re_o = Re
    c4 = -10.2192/np.log(Re)
    f_o  = 0.05273*(Re**c2)*((pd/xf)**c3)*((pf/Xt)**c4)*((np.log(A_s/A_t))**(-2.726))*(D_h/dc)**0.1325*((N_r)**0.02305)   
    G_o = rho_a*u_max

    dP_o = G_o**2/(2*rho_a)*((f_o)*(A_s/A_min_o))
    dP_fan = fan(vazao,Coef_fan[0][0],Coef_fan[0][1],Coef_fan[0][2])
    
    j_o = 1.7910*((Re)**c1)*((Xl/sigmaf)**(-0.456))*((N_r)**(-0.27))*((pf/dc)**(-1.343))*((pd/xf)**0.317)
    h_o = j_o*rho_a*u_max*cp_a*Pr_a**(-0.66666)

    #Fin efficiency

    m = ((2*h_o)/(k_w*sigmaf))**0.5

    Pm = Xl/2
    Pl = (((Xt/2)**2+Xl**2)**0.5)/2
    R_eq = 1.27*Pm*(Pl/Pm-0.3)**0.5
    phi = (R_eq/(Di/2)-1)*(1+0.35*np.log(R_eq/(Di/2)))
    eta = np.tanh(m*(Di/2)*phi)/(m*(Di/2)*phi)
    eta_o = 1 - (A_f/A_s)*(1-eta)

    Rt_o = 1/(eta_o*h_o*A_s/Nucleos)
    
    ## Internal Flow
    
    Lh=(L1+2*sigmah)*N_t/Nucleos
    A_i = np.pi*Di*L1*N_t/Nucleos
    A_min_i = np.pi*(Di**2)/4
    G_i = m_f/A_min_i
    
    Re_ic = G_i*Di/mu_wc 
    Re_ih = G_i*Di/mu_wh
    
    A_c = (2.457*np.log(1/((7/Re_ic)**0.9+0.27*(0.0015*10**(-3)/Di))))**16
    B_c = (37530/Re_ic)**16
    
    A_h = (2.457*np.log(1/((7/Re_ih)**0.9+0.27*(0.0015*10**(-3)/Di))))**16
    B_h = (37530/Re_ih)**16
    
    f_c = (8*((8/Re_ic)**12 + (A_c + B_c)**(-3/2))**(1/12))/4
    f_h = (8*((8/Re_ih)**12 + (A_h + B_h)**(-3/2))**(1/12))/4
    
    
    
    if Re_ih < 2300:
        Nu_ih = 3.66
    else:
        Nu_ih = ((f_h/2)*(Re_ih-1000)*Pr_wh)/(1+12.7*((f_h/2)**0.5)*(Pr_wh**(2/3)-1))
                                              
    if Re_ic < 2300:
        Nu_ic = 3.66
    else:
        Nu_ic = ((f_c/2)*(Re_ic-1000)*Pr_wc)/(1+12.7*((f_c/2)**0.5)*(Pr_wc**(2/3)-1))   
    
       
    
    h_i_c = Nu_ic*k_wc/Di
    h_i_h = Nu_ih*k_wh/Di
        
    Rt_i_c = 1/(h_i_c*A_i)
    Rt_i_h = 1/(h_i_h*A_i)
    
    K_bend = 50
    
    
    dP_i_c = f_c*G_i**2*4*(Lh/Di + (N_t/Nucleos)*K_bend)/(2*rho_w)
    dP_i_h = f_h*G_i**2*4*(Lh/Di + (N_t/Nucleos)*K_bend)/(2*rho_w)
    
    #http://hydroneng.com/pipelines/
    #http://www.thermopedia.com/content/577/
    #print(200*f_c,np.pi*Xt/2,Lh)
                                              
    W_p_c = dP_i_c*m_f/rho_w
    W_p_h = dP_i_h*m_f/rho_w
    
    R_tub = np.log(De/Di)/(2*np.pi*L1*N_t/Nucleos*k_w)
    
    
    UA_h = (Rt_o + R_tub + Rt_i_h)**-1
    UA_c = (Rt_o + R_tub + Rt_i_c)**-1
    
       
    eta_fan = fan(vazao,Coef_eta[0][0],Coef_eta[0][1],Coef_eta[0][2])
    W_o = vazao*dP_o/eta_fan
    
    NTU_c = UA_c/(vazao/Nucleos*rho_a*cp_a)
    NTU_h = UA_h/(vazao/Nucleos*rho_a*cp_a)
    
    C_r = (vazao/Nucleos*rho_a*cp_a)/(m_f*cp_w)
    
    if N_r == 1:
        Eff_h = (1-np.exp(-C_r*(1-np.exp(-NTU_h))))/C_r
        Eff_c = (1-np.exp(-C_r*(1-np.exp(-NTU_c))))/C_r
    elif N_r == 2:
        K = (1-np.exp(-NTU_h/2))
        Eff_h = (1-np.exp(-2*K*C_r)*((1+C_r*K**2)))/C_r
        K = (1-np.exp(-NTU_c/2))
        Eff_c = (1-np.exp(-2*K*C_r)*((1+C_r*K**2)))/C_r
    elif N_r == 3:
        K = (1-np.exp(-NTU_h/3))
        Eff_h = (1-np.exp(-3*K*C_r)*(1+C_r*K**2*(3-K)+3*C_r**2*K**4/2))/C_r
        K = (1-np.exp(-NTU_c/3))
        Eff_c = (1-np.exp(-3*K*C_r)*(1+C_r*K**2*(3-K)+3*C_r**2*K**4/2))/C_r
    elif N_r ==4:
        K = (1-np.exp(-NTU_h/4))
        Eff_h = (1-np.exp(-4*K*C_r)*(1+C_r*K**2*(6-4*K+K**2)+4*C_r**2*K**4*(2-K)+8*C_r**3*K**6/3))/C_r
        K = (1-np.exp(-NTU_c/4))
        Eff_c = (1-np.exp(-4*K*C_r)*(1+C_r*K**2*(6-4*K+K**2)+4*C_r**2*K**4*(2-K)+8*C_r**3*K**6/3))/C_r
    else:
        Eff_h = 1 - np.exp(NTU_h**0.22*(np.exp(-C_r*NTU_h**0.78)-1)/C_r)
        Eff_c = 1 - np.exp(NTU_c**0.22*(np.exp(-C_r*NTU_c**0.78)-1)/C_r) 
    
    ECr_c = Eff_c*C_r
    ECr_h = Eff_h*C_r
    
    
    m_fin = Beta*L1*(L2*L3/np.cos(theta)-np.pi*De**2*N_t)*sigmaf*rho_al
    m_hea = 2*(L2*L3-np.pi*De**2*N_t)*sigmah*rho_al
    m_tub = (np.pi*(De**2-Di**2)/4*(L1 + np.pi/2*(Xt**2+Xl**2)**0.5)*N_t)*rho_cu
    m_w   = (np.pi*Di**2/4*(L1+np.pi/2*(Xt**2+Xl**2)**0.5)*N_t)*rho_w

    return(ECr_c,ECr_h,Eff_c,Eff_h,C_r,NTU_c,NTU_h,W_p_c*Nucleos,W_p_h*Nucleos,W_o,dP_o,vazao*3600,Re_o,f_o,j_o,eta_o,eta_fan,m_fin,m_hea,m_tub,m_w,dP_i_c,dP_i_h) 

In [ ]:
def HEx_MOO(Data_Master):
    
    Xl,Xt,N_r,N_l,L1,Di,pf,Nucleos = Data_Master
    
    p_d = 0.0018
    sigmah = 0.00030 
    sigmaf = 0.15*10**-3
    xf = 7*10**-3
    De = (Di + 0.0008)
    m_f = m_f1/Nucleos
    Beta = 1/pf
    Nfin = Beta*L1   
    
    dc = (De + 2*sigmaf)
    theta = np.arctan(p_d/xf)

    L3 = ((N_l*Xt)/2*Nucleos)
    L2 = (N_r*Xl)
    N_t = (N_r*N_l/2*Nucleos)
    a = (((Xt - De) - (Xt - De)*sigmaf/np.cos(theta)*Beta)/2)
    b = (((Xt/2)**2+Xl**2)**0.5 - De - (Xt-De)*sigmaf/np.cos(theta)*Beta)
    
    
    c = np.zeros(len(a))
    for i in range(len(a)):
        if a[i]<b[i]:
            c[i] = 2*a[i]
        else:
            c[i] = 2*b[i]
    A_min_o = ((L3/Xt-1)*c + (Xt-De)-(Xt-De)*sigmaf/np.cos(theta)*Beta)*L1 #Minimum free flow area
    
    A_fr = L1*L3 #Frontal area
    
    sigma = A_min_o/A_fr
    
    A_p = np.pi*De*(L1-sigmaf*Beta*L1/np.cos(theta))*N_t+2*(L2*L3 - np.pi*De**2*N_t/4) #Primary surface area  - exposed tubes and header plates
    A_f = 2*(L2*L3/np.cos(theta)-(np.pi*De**2/4)*N_t)*Beta*L1 + 2*L3*sigmaf*Beta*L1/np.cos(theta)
    A_s = A_p + A_f       #Total heat transfer area
    A_t = np.pi*De*L1*N_t #External tube area
    
    sigma_o = (L3*L1 - L3*sigmaf*Beta*L1)/(L3*L1) #area concentration/expansion ratio
    D_h = 4*A_min_o*L2/A_s #Hydraulic diameter
    
    #Correlation parameters
    
    
    c1 = -0.1707 - 1.374*((Xl/sigmaf)**(-0.493))*((pf/dc)**(-0.886))*((N_r)**(-0.143))*((p_d/xf)**(-0.0296))
    c2 = 0.1714 - 0.07372*((pf/Xl)**0.25)*(np.log((A_s/A_t)))*((p_d/xf)**(-0.2))
    c3 = 0.426*((pf/Xt)**0.3)*(np.log((A_s/A_t)))    

    #Pressure drop correlation
    def PressureDrop(vazao):
        u_air = vazao/(L1*L3)
        u_max = u_air/sigma
        Re = (u_max*rho_a*dc/(mu_a)).astype('float64')
        c4 = -10.2192/np.log(Re)
        f_o  = 0.05273*(Re**c2)*((p_d/xf)**c3)*((pf/Xt)**c4)*((np.log((A_s/A_t)))**(-2.726))*(D_h/dc)**0.1325*((N_r)**0.02305)   
        G_o = rho_a*u_max

        dP_o = G_o**2/(2*rho_a)*((f_o)*(A_s/A_min_o))
        dP_fan = fan(np.array(list(vazao)),Coef_fan[0][0],Coef_fan[0][1],Coef_fan[0][2])
        
        return(dP_o-dP_fan)
    
    vazao = fsolve(PressureDrop,3000/3600*np.ones(len(a)))
    u_air = vazao/(L1*L3)
    u_max = u_air/sigma
    Re = u_max*rho_a*dc/(mu_a)
    Re_o = Re
    c4 = -10.2192/np.log(Re)
    f_o  = 0.05273*(Re**c2)*((p_d/xf)**c3)*((pf/Xt)**c4)*((np.log((A_s/A_t)))**(-2.726))*(D_h/dc)**0.1325*((N_r)**0.02305)   
    G_o = rho_a*u_max

    dP_o = G_o**2/(2*rho_a)*((f_o)*(A_s/A_min_o))
    dP_fan = fan(vazao,Coef_fan[0][0],Coef_fan[0][1],Coef_fan[0][2])
    
    j_o = 1.7910*((Re)**c1)*((Xl/sigmaf)**(-0.456))*((N_r)**(-0.27))*((pf/dc)**(-1.343))*((p_d/xf)**0.317)
    h_o = j_o*rho_a*u_max*cp_a*Pr_a**(-0.66666)

    #Fin efficiency

    m = ((2*h_o)/(k_w*sigmaf))**0.5

    Pm = Xl/2
    Pl = (((Xt/2)**2+Xl**2)**0.5)/2
    R_eq = 1.27*Pm*(Pl/Pm-0.3)**0.5
    phi = (R_eq/(Di/2)-1)*(1+0.35*np.log(R_eq/(Di/2)))
    eta = np.tanh(m*(Di/2)*phi)/(m*(Di/2)*phi)
    eta_o = 1 - (A_f/A_s)*(1-eta)

    Rt_o = 1/(eta_o*h_o*A_s/Nucleos)
    
    ## Internal Flow
    
    Lh=(L1+2*sigmah)*N_t/Nucleos
    A_i = np.pi*Di*L1*N_t/Nucleos
    A_min_i = np.pi*(Di**2)/4
    G_i = m_f/A_min_i
    
    Re_ic = G_i*Di/mu_wc 
    Re_ih = G_i*Di/mu_wh
    
    A_c = (2.457*np.log(1/((7/Re_ic)**0.9+0.27*(0.0015*10**(-3)/Di))))**16
    B_c = (37530/Re_ic)**16
    
    A_h = (2.457*np.log(1/((7/Re_ih)**0.9+0.27*(0.0015*10**(-3)/Di))))**16
    B_h = (37530/Re_ih)**16
    
    f_c = (8*((8/Re_ic)**12 + (A_c + B_c)**(-3/2))**(1/12))/4
    f_h = (8*((8/Re_ih)**12 + (A_h + B_h)**(-3/2))**(1/12))/4
    
    
    
    Nu_ih = np.zeros(len(Re_ih))
    Nu_ic = np.zeros(len(Re_ic))
    
    for i in range(len(Re_ih)):
        if Re_ih[i] < 2300:
            Nu_ih[i] = 3.66
        else:
            Nu_ih[i] = ((f_h[i]/2)*(Re_ih[i]-1000)*Pr_wh)/(1+12.7*((f_h[i]/2)**0.5)*(Pr_wh**(2/3)-1))

        if Re_ic[i] < 2300:
            Nu_ic[i] = 3.66
        else:
            Nu_ic[i] = ((f_c[i]/2)*(Re_ic[i]-1000)*Pr_wc)/(1+12.7*((f_c[i]/2)**0.5)*(Pr_wc**(2/3)-1))   
    
       
    
    h_i_c = Nu_ic*k_wc/Di
    h_i_h = Nu_ih*k_wh/Di
        
    Rt_i_c = 1/(h_i_c*A_i)
    Rt_i_h = 1/(h_i_h*A_i)
    
    K_bend = 50
    
    
    dP_i_c = f_c*G_i**2*4*(Lh/Di + (N_t/Nucleos)*K_bend)/(2*rho_w)
    dP_i_h = f_h*G_i**2*4*(Lh/Di + (N_t/Nucleos)*K_bend)/(2*rho_w)
    
    #http://hydroneng.com/pipelines/
    #http://www.thermopedia.com/content/577/
    #print(200*f_c,np.pi*Xt/2,Lh)
                                              
    W_p_c = dP_i_c*m_f/rho_w
    W_p_h = dP_i_h*m_f/rho_w
    
    R_tub = np.log(De/Di)/(2*np.pi*L1*N_t/Nucleos*k_w)
    
    
    UA_h = (Rt_o + R_tub + Rt_i_h)**-1
    UA_c = (Rt_o + R_tub + Rt_i_c)**-1
    
       
    eta_fan = fan(vazao,Coef_eta[0][0],Coef_eta[0][1],Coef_eta[0][2])
    W_o = vazao*dP_o/eta_fan
    
    NTU_c = UA_c/(vazao/Nucleos*rho_a*cp_a)
    NTU_h = UA_h/(vazao/Nucleos*rho_a*cp_a)
    
    C_r = (vazao/Nucleos*rho_a*cp_a)/(m_f*cp_w)
    
    Eff_h = np.zeros(len(C_r))
    Eff_c = np.zeros(len(C_r))
   

    for i in range(len(Eff_h)):
        if N_r[i] == 1:
            Eff_h[i] = (1-np.exp(-C_r[i]*(1-np.exp(-NTU_h[i]))))/C_r[i]
            Eff_c[i] = (1-np.exp(-C_r[i]*(1-np.exp(-NTU_c[i]))))/C_r[i]
        elif N_r[i] == 2:
            K = (1-np.exp(-NTU_h[i]/2))
            Eff_h[i] = (1-np.exp(-2*K*C_r[i])*((1+C_r[i]*K**2)))/C_r[i]
            K = (1-np.exp(-NTU_c[i]/2))
            Eff_c[i] = (1-np.exp(-2*K*C_r[i])*((1+C_r[i]*K**2)))/C_r[i]
        elif N_r[i] == 3:
            K = (1-np.exp(-NTU_h[i]/3))
            Eff_h[i] = (1-np.exp(-3*K*C_r[i])*(1+C_r[i]*K**2*(3-K)+3*C_r[i]**2*K**4/2))/C_r[i]
            K = (1-np.exp(-NTU_c[i]/3))
            Eff_c[i] = (1-np.exp(-3*K*C_r[i])*(1+C_r[i]*K**2*(3-K)+3*C_r[i]**2*K**4/2))/C_r[i]
        elif N_r[i] ==4:
            K = (1-np.exp(-NTU_h[i]/4))
            Eff_h[i] = (1-np.exp(-4*K*C_r[i])*(1+C_r[i]*K**2*(6-4*K+K**2)+4*C_r[i]**2*K**4*(2-K)+8*C_r[i]**3*K**6/3))/C_r[i]
            K = (1-np.exp(-NTU_c[i]/4))
            Eff_c[i] = (1-np.exp(-4*K*C_r[i])*(1+C_r[i]*K**2*(6-4*K+K**2)+4*C_r[i]**2*K**4*(2-K)+8*C_r[i]**3*K**6/3))/C_r[i]
        else:
            Eff_h[i] = 1 - np.exp(NTU_h[i]**0.22*(np.exp(-C_r[i]*NTU_h[i]**0.78)-1)/C_r[i])
            Eff_c[i] = 1 - np.exp(NTU_c[i]**0.22*(np.exp(-C_r[i]*NTU_c[i]**0.78)-1)/C_r[i]) 

    ECr_c = Eff_c*C_r
    ECr_h = Eff_h*C_r
    
    
    m_fin = Beta*L1*(L2*L3/np.cos(theta)-np.pi*De**2*N_t)*sigmaf*rho_al
    m_hea = 2*(L2*L3-np.pi*De**2*N_t)*sigmah*rho_al
    m_tub = (np.pi*(De**2-Di**2)/4*(L1 + np.pi/2*(Xt**2+Xl**2)**0.5)*N_t)*rho_cu
    m_w   = (np.pi*Di**2/4*(L1+np.pi/2*(Xt**2+Xl**2)**0.5)*N_t)*rho_w
    return(ECr_c, W_p_c*Nucleos + W_o, dP_o,L3,Eff_c,C_r,NTU_c) 

In [ ]:
class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=8,
                         n_obj=2,
                         n_constr=4,
                         #xl=np.array([30*10**-3, 31.75*10**-3, 2, 2, 0.35, 0 , 3*10**-3, 3]),
                         #xu=np.array([33*10**-3  , 38.1*10**-3 , 4, 4, 0.55, 1 , 6*10**-3, 8]))
                         xl=np.array([30*10**-3, 31.75*10**-3, 2, 1, 0.35, 0 , 4 , 3]),
                         xu=np.array([33*10**-3, 38.1*10**-3 , 8, 4, 0.55, 1 , 9 , 8]))

    def _evaluate(self, x, out, *args, **kwargs):
        #Xl,Xt,N_r,N_l,L1,Di,pf,Nucleos
        Data_MOO = np.array([x[:,0],x[:,1],x[:,2],2*x[:,3],x[:,4],0.0117 + 0.0031*x[:,5],25.4/x[:,6]*10**-3,x[:,7]],dtype='float64')
        A,B,C,D,E,F,G = HEx_MOO(Data_MOO)
        
        
        f1 = -A
        f2 = B
        
        g1 = C - 80
        g2 = -B
        g4 = 0.3 - D
        g3 = D - 0.55

        out["F"] = np.column_stack([f1, f2])
        out["G"] = np.column_stack([g1,g2,g3,g4])


problem = MyProblem()

In [ ]:
termination = MultiObjectiveSpaceToleranceTermination(tol=0.01,
                                                      n_last=500,
                                                      nth_gen=10,
                                                      n_max_gen=None,
                                                      n_max_evals=None)

In [ ]:
mask = ["real","real","int","int","real","int","int","int"]

In [ ]:
sampling = MixedVariableSampling(mask, {
    "real": get_sampling("real_random"),
    "real": get_sampling("real_random"),
    "int": get_sampling("int_random"),
    "int": get_sampling("int_random"),
    "real": get_sampling("real_random"),
    "int": get_sampling("int_random"),
    "int": get_sampling("int_random"),
    "int": get_sampling("int_random")
    
})

crossover = MixedVariableCrossover(mask, {
    "real": get_crossover("real_sbx", prob=0.9, eta=15.0),
    "real": get_crossover("real_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "real": get_crossover("real_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0)

})

mutation = MixedVariableMutation(mask, {
    "real": get_mutation("real_pm", eta=20.0),
    "real": get_mutation("real_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0),
    "real": get_mutation("real_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0) ,
    "int": get_mutation("int_pm", eta=20.0)    
})

In [ ]:
algorithm = NSGA2(
    pop_size=40,
    n_offsprings=500,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True
)

In [ ]:
p_d = 0.0018
sigmah = 0.00030 
sigmaf = 0.15*10**-3
xf = 7*10**-3

In [ ]:
def Opt():
    res = minimize(problem,
                   algorithm,
                   termination,
                   seed=1,
                   pf=problem.pareto_front(use_cache=False),
                   save_history=False,
                   verbose=False)
    
    Data = np.array([(res.X.T[0]).tolist(),
        (res.X.T[1]).tolist(),
        (res.X.T[2]).tolist(),
        (2*res.X.T[3]).tolist(),
        (res.X.T[4]).tolist(),
        (0.0117 + 0.0031*res.X.T[5]).tolist(),
        (25.4/res.X.T[6]*10**-3).tolist(),
        xf*np.ones(len(res.X.T[0])),
        sigmaf*np.ones(len(res.X.T[0])),
        sigmah*np.ones(len(res.X.T[0])),
        p_d*np.ones(len(res.X.T[0])),
        m_f1*np.ones(len(res.X.T[0])),
        (res.X.T[7]).tolist()]).T
    return res.F.T,Data

In [ ]:
mfr = np.arange(1800/3600,5450/3600,30/3600)
#mfr = [5000/360]

In [ ]:
ECr_MOO = np.array([0])
W_MOO = np.array([0])
MFR = np.array([0])
Data = [np.array([0]*13),np.array([0]*13)]
for i in range(len(mfr)):
    print(mfr[i]*3600)
    m_f1 = mfr[i]
    Res, Data_MOO = Opt()
    ECr_MOO = np.hstack((ECr_MOO,-Res[0]))
    W_MOO = np.hstack((W_MOO,Res[1]))
    MFR = np.hstack((MFR,mfr[i]*np.ones(len(Res[0]))))
    Data = np.concatenate((Data,Data_MOO))

In [ ]:
Nucleos = np.array([8])
Di = np.array([0.0117])
m_f = np.arange(4800/3600,5405/3600,50000/3600)
Xt = np.arange(31.75*10**-3,1.01*38.1*10**-3,(38.1*10**-3-31.75*10**-3)/4)
Xl = np.arange(33*10**-3,1.01*33.001*10**-3,(33*10**-3-27.5*10**-3)/4)
pf = np.array([25.4/8])*10**-3
N_r = np.arange(8,8.1,2)
N_l = np.array([4])
L1  = np.arange(0.55,0.56,0.1) #np.arange(0.4,0.73,0.05)
xf = 7*10**-3

In [ ]:
A = np.meshgrid(Xl,Xt,N_r,N_l,L1,Di,pf,xf,sigmaf,sigmah,p_d,m_f,Nucleos)
B = (np.array(A))
Data = B.T.reshape(-1,len(B))

Data = np.array([(res.X.T[0]).tolist(),
        (res.X.T[1]).tolist(),
        (2*res.X.T[2]).tolist(),
        (2*res.X.T[3]).tolist(),
        (res.X.T[4]).tolist(),
        (0.0117 + 0.0031*res.X.T[5]).tolist(),
        (res.X.T[6]).tolist(),
        xf*np.ones(len(res.X.T[0])),
        sigmaf*np.ones(len(res.X.T[0])),
        sigmah*np.ones(len(res.X.T[0])),
        p_d*np.ones(len(res.X.T[0])),
        m_f*np.ones(len(res.X.T[0])),
        (res.X.T[7]).tolist()]).T

In [ ]:
Res = (HEx(Data[0]))

In [ ]:
Res[10]

In [ ]:
#%%timeit

Res = np.zeros([len(Data),23])
Data_Final = np.zeros([len(Data),len(Data.T) + len(Res.T)])
for i in range (len(Data)):
    if  i % 1 == 0:
        print(i/(len(Data))*100)
    #if Data[i][1]*Data[i][3]*Data[i][12]/2>0.3 and Data[i][1]*Data[i][3]*Data[i][12]/2<0.55:
        Res[i] = (HEx(Data[i]))
        #if Res[i][10] < 80: #and Res[i][11] < 2548:
            #print(i)
        Data_Final[i] = np.concatenate([Data[i],Res[i]])     

In [ ]:
#m_f = mfr

In [ ]:
pd.DataFrame(Data_Final, columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])

In [ ]:
df = pd.DataFrame(Data_Final, columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])
df = df[df['m_f']>0]
df_mf = df.groupby('m_f')
df_mf.get_group(m_f[0])

In [ ]:
df.to_excel("Data_HEx_Tubal.xlsx")

In [ ]:
#df =pd.read_excel("Dissertation.xlsx", columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])

In [ ]:
Xl_1 = [[] for i in range(len(m_f))]
Xt_1 = [[] for i in range(len(m_f))]
Nr_1 = [[] for i in range(len(m_f))]
Nl_1 = [[] for i in range(len(m_f))]
L1_1 = [[] for i in range(len(m_f))]
Di_1 = [[] for i in range(len(m_f))]
pf_1 = [[] for i in range(len(m_f))]
mf_1 = [[] for i in range(len(m_f))]
Nucleos_1 = [[] for i in range(len(m_f))]
Ecrc_1 = [[] for i in range(len(m_f))]
Ecrh_1 = [[] for i in range(len(m_f))]
Efc_1 = [[] for i in range(len(m_f))]
Efh_1 = [[] for i in range(len(m_f))]
C_1 = [[] for i in range(len(m_f))]
NTUc_1 = [[] for i in range(len(m_f))]
NTUh_1 = [[] for i in range(len(m_f))]
Wpc_1 = [[] for i in range(len(m_f))]
Wph_1 = [[] for i in range(len(m_f))]
Wo_1 = [[] for i in range(len(m_f))]
dPo_1 = [[] for i in range(len(m_f))]
cfm_1 = [[] for i in range(len(m_f))]
Reo_1 = [[] for i in range(len(m_f))]
fo_1 = [[] for i in range(len(m_f))]
jo_1 = [[] for i in range(len(m_f))]
etao_1 = [[] for i in range(len(m_f))]
etaf_1 = [[] for i in range(len(m_f))]
mfin_1 = [[] for i in range(len(m_f))]
mhea_1 = [[] for i in range(len(m_f))]
mtub_1 = [[] for i in range(len(m_f))]
mwa_1 = [[] for i in range(len(m_f))]
dPc_1 = [[] for i in range(len(m_f))]
dPh_1 = [[] for i in range(len(m_f))]


ECr = [[] for i in range(len(m_f))]
W_Tc = [[] for i in range(len(m_f))]
W_Th = [[] for i in range(len(m_f))]
W_o = [[] for i in range(len(m_f))]
m_tot = [[] for i in range(len(m_f))]
Count = [[] for i in range(len(m_f))]

for i in range(len(m_f)):    
    ECr[i].append(np.array(df_mf.get_group(m_f[i])['ECr_c']))
    W_Tc[i].append(np.array(df_mf.get_group(m_f[i])['W_o']+df_mf.get_group(m_f[i])['W_p_c']))
    W_Th[i].append(np.array(df_mf.get_group(m_f[i])['W_o']+df_mf.get_group(m_f[i])['W_p_h']))
    m_tot[i].append(np.array(df_mf.get_group(m_f[i])['m_fin']+df_mf.get_group(m_f[i])['m_tub']+df_mf.get_group(m_f[i])['m_hea']+df_mf.get_group(m_f[i])['m_w']))
    W_o[i].append(np.array(df_mf.get_group(m_f[i])['W_p_c']))
    m_tot[i].append(np.array(df_mf.get_group(m_f[i])['m_fin']+df_mf.get_group(m_f[i])['m_tub']+df_mf.get_group(m_f[i])['m_hea']+df_mf.get_group(m_f[i])['m_w']))
    
    Xl_1[i].append(np.array(df_mf.get_group(m_f[i])['Xl']))
    Xt_1[i].append(np.array(df_mf.get_group(m_f[i])['Xt']))
    Nr_1[i].append(np.array(df_mf.get_group(m_f[i])['N_r']))
    Nl_1[i].append(np.array(df_mf.get_group(m_f[i])['N_l']))
    L1_1[i].append(np.array(df_mf.get_group(m_f[i])['L1']))
    Di_1[i].append(np.array(df_mf.get_group(m_f[i])['Di']))
    pf_1[i].append(np.array(df_mf.get_group(m_f[i])['pf']))
    mf_1[i].append(np.array(df_mf.get_group(m_f[i])['m_f']))
    Nucleos_1[i].append(np.array(df_mf.get_group(m_f[i])['Nucleos']))
    Ecrc_1[i].append(np.array(df_mf.get_group(m_f[i])['ECr_c']))
    Ecrh_1[i].append(np.array(df_mf.get_group(m_f[i])['ECr_h']))
    Efc_1[i].append(np.array(df_mf.get_group(m_f[i])['Eff_c']))
    Efh_1[i].append(np.array(df_mf.get_group(m_f[i])['Eff_h']))
    C_1[i].append(np.array(df_mf.get_group(m_f[i])['C_r']))
    NTUc_1[i].append(np.array(df_mf.get_group(m_f[i])['NTU_c']))
    NTUh_1[i].append(np.array(df_mf.get_group(m_f[i])['NTU_h']))
    Wpc_1[i].append(np.array(df_mf.get_group(m_f[i])['W_p_c']))
    Wph_1[i].append(np.array(df_mf.get_group(m_f[i])['W_p_h']))
    Wo_1[i].append(np.array(df_mf.get_group(m_f[i])['W_o']))
    dPo_1[i].append(np.array(df_mf.get_group(m_f[i])['dP_o']))
    cfm_1[i].append(np.array(df_mf.get_group(m_f[i])['vazao*3600']))
    Reo_1[i].append(np.array(df_mf.get_group(m_f[i])['Re_o']))
    fo_1[i].append(np.array(df_mf.get_group(m_f[i])['f_o']))
    jo_1[i].append(np.array(df_mf.get_group(m_f[i])['j_o']))
    etao_1[i].append(np.array(df_mf.get_group(m_f[i])['eta_o']))
    etaf_1[i].append(np.array(df_mf.get_group(m_f[i])['eta_fan']))
    mfin_1[i].append(np.array(df_mf.get_group(m_f[i])['m_fin']))
    mhea_1[i].append(np.array(df_mf.get_group(m_f[i])['m_hea']))
    mtub_1[i].append(np.array(df_mf.get_group(m_f[i])['m_tub']))
    mwa_1[i].append(np.array(df_mf.get_group(m_f[i])['m_w']))
    dPc_1[i].append(np.array(df_mf.get_group(m_f[i])['dP_c']))
    dPh_1[i].append(np.array(df_mf.get_group(m_f[i])['dP_h']))
    

In [ ]:
Cont = [[] for i in range(len(m_f))]
Ec_p_L = [[] for i in range(len(m_f))]
W_T_pc =  [[] for i in range(len(m_f))]
cfm_p_L = [[] for i in range(len(m_f))]

for k in range(np.size(m_f)):
    A = np.zeros(len(ECr[k][0]))
    for i in range(len(ECr[k][0])):
        for j in range(len(ECr[k][0])):
            #if ECr[k][0][i] < ECr[k][0][j] and cfm_1[k][0][i] > cfm_1[k][0][j]:
            if ECr[k][0][i] < ECr[k][0][j] and W_Tc[k][0][i] > W_Tc[k][0][j]:
                A[i] = 1
                break
        if A[i] == 0:
            Cont[k].append(i)
            Ec_p_L[k].append(ECr[k][0][i])
            W_T_pc[k].append(W_Tc[k][0][i])
            #cfm_p_L[k].append(cfm_1[k][0][i])
            #print(Cont[k],Ec_p_L[k],W_p_L[k])
    print(k/len(m_f)*100)

In [ ]:
Xl_p = []
Xt_p = []
Nr_p = []
Nl_p = []
L1_p = []
Di_p = []
pf_p = []
mf_p = []
Nucleos_p = []
Ecrc_p = []
Ecrh_p = []
Efc_p = []
Efh_p = []
C_p = []
NTUc_p = []
NTUh_p = []
Wpc_p = []
Wph_p = []
Wo_p = []
dPo_p = []
cfm_p = []
Reo_p = []
fo_p = []
jo_p = []
etao_p = []
etaf_p = []
mfin_p = []
mhea_p = []
mtub_p = []
mwa_p = []
dPc_p = []
dPh_p = []
    
    
for k in range(np.size(m_f)):
    for i in range(len(Cont[k])):
        Xl_p.append(Xl_1[k][0][Cont[k][i]])
        Xt_p.append(Xt_1[k][0][Cont[k][i]])
        Nr_p.append(Nr_1[k][0][Cont[k][i]])
        Nl_p.append(Nl_1[k][0][Cont[k][i]])
        L1_p.append(L1_1[k][0][Cont[k][i]])
        Di_p.append(Di_1[k][0][Cont[k][i]])
        pf_p.append(pf_1[k][0][Cont[k][i]])
        mf_p.append(mf_1[k][0][Cont[k][i]])
        Nucleos_p.append(Nucleos_1[k][0][Cont[k][i]])
        Ecrc_p.append(Ecrc_1[k][0][Cont[k][i]])
        Ecrh_p.append(Ecrh_1[k][0][Cont[k][i]])
        Efc_p.append(Efc_1[k][0][Cont[k][i]])
        Efh_p.append(Efh_1[k][0][Cont[k][i]])
        C_p.append(C_1[k][0][Cont[k][i]])
        NTUc_p.append(NTUc_1[k][0][Cont[k][i]])
        NTUh_p.append(NTUh_1[k][0][Cont[k][i]])
        Wpc_p.append(Wpc_1[k][0][Cont[k][i]])
        Wph_p.append(Wph_1[k][0][Cont[k][i]])
        Wo_p.append(Wo_1[k][0][Cont[k][i]])
        dPo_p.append(dPo_1[k][0][Cont[k][i]])
        cfm_p.append(cfm_1[k][0][Cont[k][i]])
        Reo_p.append(Reo_1[k][0][Cont[k][i]])
        fo_p.append(fo_1[k][0][Cont[k][i]])
        jo_p.append(jo_1[k][0][Cont[k][i]])
        etao_p.append(etao_1[k][0][Cont[k][i]])
        etaf_p.append(etaf_1[k][0][Cont[k][i]])
        mfin_p.append(mfin_1[k][0][Cont[k][i]])
        mhea_p.append(mhea_1[k][0][Cont[k][i]])
        mtub_p.append(mtub_1[k][0][Cont[k][i]])
        mwa_p.append(mwa_1[k][0][Cont[k][i]])
        dPc_p.append(dPc_1[k][0][Cont[k][i]])
        dPh_p.append(dPh_1[k][0][Cont[k][i]])

In [ ]:
plt.plot(mf_p,np.array(cfm_p)*0.588,'x')

In [ ]:
#plt.style.use('seaborn-whitegrid')
plt.style.use('default')
plt.rc('font',family = 'serif',size = 12)
plt.rc('text', usetex = True)

In [ ]:
plt.plot(Reo_p,fo_p,'x',color = '#1f77b4')
plt.plot(Reo_p,jo_p,'x',color = '#d62728')

plt.grid(linestyle = 'dashed')

In [ ]:
Data_Final = pd.DataFrame({'Xl':Xl_p,
                'Xt':Xt_p,
                'Nr':Nr_p,
                'Nl':Nl_p, 
                'L1':L1_p, 
                'Di':Di_p, 
                'pf':pf_p,
                'mf':mf_p, 
                'Nucleos':Nucleos_p, 
                'ECrC':Ecrc_p, 
                'ECrH':Ecrh_p, 
                'EffC':Efc_p, 
                'EffH':Efh_p, 
                'Cr':C_p, 
                'NTUC':NTUc_p, 
                'NTUH':NTUh_p, 
                'Wpc':Wpc_p, 
                'Wph':Wph_p, 
                'Wo':Wo_p, 
                'dPo':dPo_p,
                'vazao':cfm_p,
                'Reo':Reo_p,
                'fo':fo_p,
                'jo':jo_p,
                'etao':etao_p,
                'etaf':etaf_p,
                'mfin':mfin_p,
                'mhea':mhea_p,
                'mtub':mtub_p,
                'mwa':mwa_p,
                'dPc':dPc_p,
                'dPh':dPh_p,
             })

In [ ]:
Data_Final.to_excel("HEx_APEn_2.xlsx") 

In [ ]:
Data_Final

In [ ]:
plt.plot(np.array(Ecrc_p[0:len(Ec_p_L[0])]),np.array(pf_p[0:len(Ec_p_L[0])])*1000,'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 4800 kg/h')
plt.plot(np.array(Ecrc_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),np.array(pf_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])])*1000,'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 6400 kg/h')
plt.plot(np.array(Ecrc_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),np.array(pf_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])])*1000,'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 8000 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$p_\mathrm{fin}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#sc.set_alpha(0.75)
plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0,0.16])

plt.savefig('HEx_pf.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(np.array(Ec_p_L[0:len(Ec_p_L[0])]),Ec_p_L[0],'o',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 2500 kg/h')
plt.plot(np.array(Ec_p_L[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),Ec_p_L[1],'x',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 5000 kg/h')
plt.plot(np.array(Ec_p_L[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),Ec_p_L[2],'P',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 7500 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{V}\llap{---}_\mathrm{a}$ [cfm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#sc.set_alpha(0.5)
plt.legend(bbox_to_anchor=[0.55, 0.8])

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
plt.ylim([0.03,0.23])

#plt.savefig('HEx_Paretto.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
np.shape(np.array(W_T_pc)[0])

In [ ]:
plt.plot(np.array(W_T_pc)[0],np.array(Ec_p_L)[0],'o',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(res.F.T[1],-res.F.T[0],'o')
plt.plot(W_MOO,ECr_MOO,'x')
#plt.plot(np.array(W_T_pc)[1],np.array(Ec_p_L)[1],'x',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(np.array(W_T_pc)[2],np.array(Ec_p_L)[2],'P',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#sc.set_alpha(0.5)
plt.legend(bbox_to_anchor=[0.55, 0.8])
#plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.03,0.23])

#plt.savefig('HEx_Paretto.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')
plt.savefig('HEx_Paretto.jpg',format = 'jpg',dpi = 300,bbox_inches = 'tight')

In [ ]:
H_p = np.array(Xt_p)*np.array(Nl_p)*np.array(Nucleos_p)/2
L_p = np.array(Xl_p)*np.array(Nr_p)

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(L_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(L_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(L_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$L_\mathrm{HEx}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.28, 0.43])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.29,0.57])

#plt.savefig('HEx_L.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(H_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(H_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(H_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$H_\mathrm{HEx}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.25, 0.17])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([0.06,0.27])

plt.savefig('HEx_H.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(L1_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(L1_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(L1_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$W_\mathrm{HEx}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.25, 0.33])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([0.06,0.30])

plt.savefig('HEx_W.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(Di_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
plt.plot(Ec_p_L[1],1000*np.array(Di_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(Di_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$D_\mathrm{i}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.25, 0.33])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.29,0.57])

#plt.savefig('HEx_Di.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(pf_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(pf_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(pf_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$p_\mathrm{fin}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
plt.legend(bbox_to_anchor=[0.5, 0.4])
#plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.29,0.57])

#plt.savefig('HEx_pf.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:

plt.plot(Ec_p_L[0],(Xl_p[0:len(Ec_p_L[0])]-min(Xl_p[0:len(Ec_p_L[0])]))/(max(Xl_p[0:len(Ec_p_L[0])])-min(Xl_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Xt_p[0:len(Ec_p_L[0])]-min(Xt_p[0:len(Ec_p_L[0])]))/(max(Xt_p[0:len(Ec_p_L[0])])-min(Xt_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(L1_p[0:len(Ec_p_L[0])]-min(L1_p[0:len(Ec_p_L[0])]))/(max(L1_p[0:len(Ec_p_L[0])])-min(L1_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Nucleos_p[0:len(Ec_p_L[0])]-min(Nucleos_p[0:len(Ec_p_L[0])]))/(max(Nucleos_p[0:len(Ec_p_L[0])])-min(Nucleos_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]-min(Nr_p[0:len(Ec_p_L[0])]))/(max(Nr_p[0:len(Ec_p_L[0])])-min(Nr_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]-min(Nl_p[0:len(Ec_p_L[0])]))/(max(Nl_p[0:len(Ec_p_L[0])])-min(Nl_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(pf_p[0:len(Ec_p_L[0])]-min(pf_p[0:len(Ec_p_L[0])]))/(max(pf_p[0:len(Ec_p_L[0])])-min(pf_p[0:len(Ec_p_L[0])])),'o')


Data_Final.to_excel("Results.xlsx") 

In [ ]:
EcrC = Data_Final['ECrC']
#WTT = Data_Final['Wo'] + Data_Final['Wpc']
WTT =  Data_Final['Wpc']

In [ ]:
DataF = Data_Final.groupby('mf')
Ecr_mf = DataF.get_group(m_f[0])['ECrC']
WTT_mf = DataF.get_group(m_f[0])['Wo'] + DataF.get_group(m_f[0])['Wpc']

In [ ]:
len(WTT_mf)

In [ ]:
import matplotlib.style
import matplotlib as mpl
mpl.style.use('default')
plt.rc('text',usetex = True)
plt.rc('font', family='serif',size = 12)
cmap = ('viridis')

In [ ]:
plt.grid(linestyle = 'dashed')
plt.scatter(WTT,EcrC, alpha = 0.5, c = Data_Final['mf'],cmap = 'viridis')

#plt.scatter(WTT_mf,Ecr_mf)
#lt.plot(WTT,Ecr,'o')
plt.colorbar(label = 'Mass flow rate [kg/h]')
plt.ylabel(r'$\varepsilon C_\mathrm{r}$ [-]')
plt.xlabel('Power [W]')


In [ ]:
Wt_1 = WTT
Ec_1 = EcrC

In [ ]:
plt.semilogx(Wt_1,Ec_1,'o',label = 'N$_\mathrm{Circ}$ = 1')
plt.semilogx(Wt_2,Ec_2,'x',label = 'N$_\mathrm{Circ}$ = 2')
plt.semilogx(Wt_3,Ec_3,'s',label = 'N$_\mathrm{Circ}$ = 3')
plt.semilogx(Wt_4,Ec_4,'^',label = 'N$_\mathrm{Circ}$ = 4')
plt.semilogx(Wt_6,Ec_6,'v',label = 'N$_\mathrm{Circ}$ = 6')
#plt.semilogx(Wt_12,Ec_12,'.',label = 'N$_\mathrm{Circ}$ = 12')
plt.legend()
plt.grid(linestyle = 'dashed')
plt.xlim(0,10000)
plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx,pump}$ [W]')
plt.savefig('N_Circ.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
s1 = 'L1'
size = (5*(np.array(df_mf.get_group(m_f[0])[s1]) - min(np.array(df_mf.get_group(m_f[0])[s1])))/(max(np.array(df_mf.get_group(m_f[0])[s1])) - min(np.array(df_mf.get_group(m_f[0])[s1]))))**2



c1 = 'mf'
#col = np.array(df_mf.get_group(m_f[0])[c1])
size =  m_tot[0][0]
col = np.array(df_mf.get_group(m_f[0])[s1])

In [ ]:
DataF = Data_Final.groupby('mf')
Ecr = DataF.get_group(m_f[0])['ECrC']
WTT = DataF.get_group(m_f[0])['Wo'] + DataF.get_group(m_f[0])['Wpc']

In [ ]:
Data_Final['W_total'] = Data_Final['Wo'] + Data_Final['Wpc']

In [ ]:
Data_Final[Data_Final['W_total']<350]

In [ ]:
DataF.get_group(m_f[0])

In [ ]:
DataFi = Data_Final
DataF = DataFi.groupby('mf')

In [ ]:
#DataF = Data_Final.groupby('mf')
col  = np.array(DataF.get_group(m_f[0])['L1'])
size = np.array(DataF.get_group(m_f[0])['mfin']+DataF.get_group(m_f[0])['mhea']+DataF.get_group(m_f[0])['mwa']+DataF.get_group(m_f[0])['mtub'])
Ecr_mf = DataF.get_group(m_f[0])['ECrC']
WTT_mf = DataF.get_group(m_f[0])['Wo'] + DataF.get_group(m_f[0])['Wpc']

m_tot_1 = mfin_1+mhea_1+mtub_1+mwa_1


In [ ]:
DFF = pd.DataFrame(np.array([cfm_1[2][0],ECr[2][0],W_Tc[2][0],m_tot[2][0]]).T,
                              columns = ['cfm','ECr','W','m'])

In [ ]:
DFF['W']

In [ ]:
DFF= DFF[DFF['W']<350]


In [ ]:
i = 0
sc = plt.scatter(DFF['cfm'],DFF['ECr'],marker = 'o',cmap = 'jet', c = DFF['W'],s = DFF['m'],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')
#sc = plt.scatter(cfm_1[i][0]*0.588,ECr[i][0],marker = 'o',cmap = 'jet', c= Xl_1[i][0])#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

plt.plot(np.array(cfm_p_L[i]),Ec_p_L[i],'X',color = 'black', label = 'Pareto Front')
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]',bbox_to_anchor=[1.55, 0.45])
sc.set_alpha(0.5)
#plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0,0.16])

#plt.savefig('HEx_5400_cfm.pdf',format = 'pdf',bbox_inches='tight')
plt.savefig('HEx_5400_cfm.jpg',format = 'jpg',dpi = 300,bbox_inches = 'tight')

In [ ]:
i = 0

sc = plt.scatter(W_Tc[i][0],ECr[i][0],marker = 'o',cmap = 'jet', c = cfm_1[i][0],s = m_tot[i][0],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

#plt.plot(WTT,Ecr,'x',color = 'black', mfc='none', label = 'Pareto Front',alpha = 0.65)
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx}$ [W]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]')
sc.set_alpha(0.5)
#plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([146,170.5])
plt.savefig('HEx_2400.pdf',format = 'pdf',bbox_inches='tight')

In [ ]:
i = 1

sc = plt.scatter(W_Tc[i][0],ECr[i][0],marker = 'o',cmap = 'jet', c = cfm_1[i][0],s = m_tot[i][0],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

#plt.plot(WTT,Ecr,'x',color = 'black', mfc='none', label = 'Pareto Front',alpha = 0.65)
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx}$ [W]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]')
sc.set_alpha(0.5)
#plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([146,170.5])
plt.savefig('HEx_3900.pdf',format = 'pdf',bbox_inches='tight')

In [ ]:
i = 0

sc = plt.scatter(W_Tc[i][0],ECr[i][0],marker = 'o',cmap = 'jet', c = cfm_1[i][0],s = m_tot[i][0],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

#plt.plot(WTT,Ecr,'x',color = 'black', mfc='none', label = 'Pareto Front',alpha = 0.65)
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx}$ [W]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]')
sc.set_alpha(0.5)
#plt.legend()
#plt.xlim([0,350])
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([146,170.5])
plt.savefig('HEx_5400.pdf',format = 'pdf',bbox_inches='tight')

In [ ]:
#plt.scatter(m_tot[0][0],ECr[0][0], c = np.array(df_mf.get_group(m_f[0])['m_w']/m_tot[0][0]))
plt.scatter(m_tot[0][0],ECr[0][0], c = col)
plt.colorbar(label = 'Internal Pumping Power [W]')
plt.grid()

In [ ]:
df_final = pd.DataFrame(columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])


In [ ]:
A = [[] for i in range(len(Cont[0]))]

In [ ]:
np.array(df)[Cont[0][0]]

In [ ]:
A[1].append(np.array(df)[Cont[0][1]])

In [ ]:
A = np.array(df)[Cont[0][1]]

In [ ]:
A

In [ ]:
A.append(np.array(df)[Cont[0][0]])

In [ ]:
df_final.append(np.array(df)[Cont[0][0]])